In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.preprocessing import LabelEncoder

import pickle
import json

In [2]:
df=pd.read_csv('master.csv')
df

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z


# EDA

In [3]:
df.isna().sum()

country                   0
year                      0
sex                       0
age                       0
suicides_no               0
population                0
suicides/100k pop         0
country-year              0
HDI for year          19456
 gdp_for_year ($)         0
gdp_per_capita ($)        0
generation                0
dtype: int64

In [4]:
df.drop('HDI for year',axis=1,inplace=True)
df.isna().sum()

country               0
year                  0
sex                   0
age                   0
suicides_no           0
population            0
suicides/100k pop     0
country-year          0
 gdp_for_year ($)     0
gdp_per_capita ($)    0
generation            0
dtype: int64

In [5]:
df

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,"63,067,077,179",2309,Generation Z


In [6]:
df.isna().sum()

country               0
year                  0
sex                   0
age                   0
suicides_no           0
population            0
suicides/100k pop     0
country-year          0
 gdp_for_year ($)     0
gdp_per_capita ($)    0
generation            0
dtype: int64

In [7]:
df['generation'].value_counts()

Generation X       6408
Silent             6364
Millenials         5844
Boomers            4990
G.I. Generation    2744
Generation Z       1470
Name: generation, dtype: int64

In [8]:
df.corr()

,year,suicides_no,population,suicides/100k pop,gdp_per_capita ($)
year,1.000000,-0.004546,0.008850,-0.039037,0.339134
suicides_no,-0.004546,1.000000,0.616162,0.306604,0.061330
population,0.008850,0.616162,1.000000,0.008285,0.081510
suicides/100k pop,-0.039037,0.306604,0.008285,1.000000,0.001785
gdp_per_capita ($),0.339134,0.061330,0.081510,0.001785,1.000000


In [9]:
df['sex'].replace({'male':0,'female':1},inplace=True)

In [10]:
df

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,0,15-24 years,21,312900,6.71,Albania1987,"2,156,624,900",796,Generation X
1,Albania,1987,0,35-54 years,16,308000,5.19,Albania1987,"2,156,624,900",796,Silent
2,Albania,1987,1,15-24 years,14,289700,4.83,Albania1987,"2,156,624,900",796,Generation X
3,Albania,1987,0,75+ years,1,21800,4.59,Albania1987,"2,156,624,900",796,G.I. Generation
4,Albania,1987,0,25-34 years,9,274300,3.28,Albania1987,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,1,35-54 years,107,3620833,2.96,Uzbekistan2014,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,1,75+ years,9,348465,2.58,Uzbekistan2014,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,0,5-14 years,60,2762158,2.17,Uzbekistan2014,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,1,5-14 years,44,2631600,1.67,Uzbekistan2014,"63,067,077,179",2309,Generation Z


In [11]:
df.drop(['country','country-year'],axis=1,inplace=True)
df

,year,sex,age,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($),generation
0,1987,0,15-24 years,21,312900,6.71,"2,156,624,900",796,Generation X
1,1987,0,35-54 years,16,308000,5.19,"2,156,624,900",796,Silent
2,1987,1,15-24 years,14,289700,4.83,"2,156,624,900",796,Generation X
3,1987,0,75+ years,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,1987,0,25-34 years,9,274300,3.28,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...
27815,2014,1,35-54 years,107,3620833,2.96,"63,067,077,179",2309,Generation X
27816,2014,1,75+ years,9,348465,2.58,"63,067,077,179",2309,Silent
27817,2014,0,5-14 years,60,2762158,2.17,"63,067,077,179",2309,Generation Z
27818,2014,1,5-14 years,44,2631600,1.67,"63,067,077,179",2309,Generation Z


In [12]:
df['age'].value_counts().to_dict()

{'15-24 years': 4642,
 '35-54 years': 4642,
 '75+ years': 4642,
 '25-34 years': 4642,
 '55-74 years': 4642,
 '5-14 years': 4610}

In [13]:
df['age'].replace({'15-24 years': 0,
 '35-54 years': 1,
 '75+ years': 2,
 '25-34 years': 3,
 '55-74 years': 4,
 '5-14 years': 5},inplace=True)
df

,year,sex,age,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($),generation
0,1987,0,0,21,312900,6.71,"2,156,624,900",796,Generation X
1,1987,0,1,16,308000,5.19,"2,156,624,900",796,Silent
2,1987,1,0,14,289700,4.83,"2,156,624,900",796,Generation X
3,1987,0,2,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,1987,0,3,9,274300,3.28,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...
27815,2014,1,1,107,3620833,2.96,"63,067,077,179",2309,Generation X
27816,2014,1,2,9,348465,2.58,"63,067,077,179",2309,Silent
27817,2014,0,5,60,2762158,2.17,"63,067,077,179",2309,Generation Z
27818,2014,1,5,44,2631600,1.67,"63,067,077,179",2309,Generation Z


In [14]:
lst=[]
for i in df[' gdp_for_year ($) ']:
    m=i.replace(',','')
    m=int(m)
    lst.append(m)
    len(lst)


In [15]:
df['gdp_for_year_($)']=lst

In [16]:
df.drop(' gdp_for_year ($) ',axis=1,inplace=True)
df

,year,sex,age,suicides_no,population,suicides/100k pop,gdp_per_capita ($),generation,gdp_for_year_($)
0,1987,0,0,21,312900,6.71,796,Generation X,2156624900
1,1987,0,1,16,308000,5.19,796,Silent,2156624900
2,1987,1,0,14,289700,4.83,796,Generation X,2156624900
3,1987,0,2,1,21800,4.59,796,G.I. Generation,2156624900
4,1987,0,3,9,274300,3.28,796,Boomers,2156624900
...,...,...,...,...,...,...,...,...,...
27815,2014,1,1,107,3620833,2.96,2309,Generation X,63067077179
27816,2014,1,2,9,348465,2.58,2309,Silent,63067077179
27817,2014,0,5,60,2762158,2.17,2309,Generation Z,63067077179
27818,2014,1,5,44,2631600,1.67,2309,Generation Z,63067077179


In [17]:

x=df.drop('generation',axis=1)
y=df['generation']
le=LabelEncoder()
y=le.fit_transform(y)


In [18]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,stratify=y,random_state=10)

In [19]:
y_train

array([1, 3, 5, ..., 4, 3, 0])

In [20]:
x_train

,year,sex,age,suicides_no,population,suicides/100k pop,gdp_per_capita ($),gdp_for_year_($)
16235,1989,0,4,8,47500,16.84,2349,2191096860
7370,2012,0,5,3,495721,0.61,20898,207376427021
11008,2011,0,2,5,117667,4.25,3661,47654841113
7961,2007,0,2,18,145654,12.36,4024,51007777000
6258,2005,0,5,2,410294,0.49,5133,19952156475
...,...,...,...,...,...,...,...,...
18787,2006,1,1,18,565035,3.19,2038,10646157920
12628,2002,1,0,14,540419,2.59,20536,121069378150
26785,2010,1,0,106,3995780,2.65,41798,2441173394730
19019,2007,1,5,13,10163198,0.13,1915,149359918060


In [21]:
x_train['suicide_rate']=x_train['suicides/100k pop']
x_test['suicide_rate']=x_test['suicides/100k pop']

In [22]:
x_train

,year,sex,age,suicides_no,population,suicides/100k pop,gdp_per_capita ($),gdp_for_year_($),suicide_rate
16235,1989,0,4,8,47500,16.84,2349,2191096860,16.84
7370,2012,0,5,3,495721,0.61,20898,207376427021,0.61
11008,2011,0,2,5,117667,4.25,3661,47654841113,4.25
7961,2007,0,2,18,145654,12.36,4024,51007777000,12.36
6258,2005,0,5,2,410294,0.49,5133,19952156475,0.49
...,...,...,...,...,...,...,...,...,...
18787,2006,1,1,18,565035,3.19,2038,10646157920,3.19
12628,2002,1,0,14,540419,2.59,20536,121069378150,2.59
26785,2010,1,0,106,3995780,2.65,41798,2441173394730,2.65
19019,2007,1,5,13,10163198,0.13,1915,149359918060,0.13


In [23]:
x_train.drop(['suicides/100k pop','suicides/100k pop'],axis=1,inplace=True)
x_test.drop(['suicides/100k pop','suicides/100k pop'],axis=1,inplace=True)

In [24]:
x_train['gdp_per_capita_($)']=x_train['gdp_per_capita ($)']

In [25]:
x_test['gdp_per_capita_($)']=x_test['gdp_per_capita ($)']

In [26]:
print(x_train.columns)
print(x_test.columns)

Index(['year', 'sex', 'age', 'suicides_no', 'population', 'gdp_per_capita ($)',
       'gdp_for_year_($)', 'suicide_rate', 'gdp_per_capita_($)'],
      dtype='object')
Index(['year', 'sex', 'age', 'suicides_no', 'population', 'gdp_per_capita ($)',
       'gdp_for_year_($)', 'suicide_rate', 'gdp_per_capita_($)'],
      dtype='object')


In [27]:
x_train.drop('gdp_per_capita ($)',axis=1,inplace=True)
x_test.drop('gdp_per_capita ($)',axis=1,inplace=True)

In [28]:
model=RandomForestClassifier()

In [29]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [30]:
model.predict(x_test)

array([2, 5, 0, ..., 0, 4, 5])

In [31]:
y_test

array([2, 5, 0, ..., 0, 4, 5])


# Model Evaluation

In [32]:
#training Model
acc_train=model.score(x_train,y_train)
print(f'Train Acc={acc_train}')
acc_test=model.score(x_test,y_test)
print(f'Test Acc={acc_test}')

Train Acc=1.0
Test Acc=0.9991612748622094


In [33]:
with open('model_master.pkl','wb') as file: 
    pickle.dump(model,file)

In [34]:
x_train.columns.to_list()

['year',
 'sex',
 'age',
 'suicides_no',
 'population',
 'gdp_for_year_($)',
 'suicide_rate',
 'gdp_per_capita_($)']

In [35]:
year=int(input('Enter Year'))
sex=int(input('Enter Sex')) ##Radio
age=int(input('Enter Age'))  #Radio
suicides_no=int(input('Enter Suicide No.'))
population=int(input('Enter Population'))
gdp_for_year=int(input('Enter GDP per year'))
suicide_rate=float(input('Enter Suicide Rate'))
gdp_per_capita=int(input('Enter GDP per Capital'))



data=np.zeros(len(x.columns))
data[0]=year
data[1]=sex
data[2]=age
data[3]=suicides_no
data[4]=population
data[7]=gdp_per_capita
data[5]=gdp_for_year
data[6]=suicide_rate



# 15-24 years=0,
# 35-54 years= 1,
# 75+ years=2,
# 25-34 years= 3,
# 55-74 years= 4,
# 5-14 years= 5
pred=model.predict(data.reshape(1,8))
out=['Boomers', 'G.I. Generation', 'Generation X', 'Generation Z','Millenials', 'Silent']
if pred==0:
    print(out[0])
elif pred==1:
    print(out[1])
elif pred==2:
    print(out[2])
elif pred==3:
    print(out[3])
elif pred==4:
    print(out[4])
elif pred==5:
    print(out[5])

    


KeyboardInterrupt



In [36]:
df[df['population']==11313100]

,year,sex,age,suicides_no,population,suicides/100k pop,gdp_per_capita ($),generation,gdp_for_year_($)
9748,1993,0,1,3033,11313100,26.81,26937,Boomers,2068555542411
9760,1994,0,1,3154,11313100,27.88,28588,Boomers,2205966011812


In [97]:
le.classes_

array(['Boomers', 'G.I. Generation', 'Generation X', 'Generation Z',
       'Millenials', 'Silent'], dtype=object)